# Step 1: Decode the InsuranceQA Dataset

In [1]:
import json
import gzip

# Function to load vocabulary from a file
def load_vocabulary(vocab_file_path):
    vocab_dict = {}
    with open(vocab_file_path, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t", 1)  # Split only on the first tab
            if len(parts) == 2:
                index, word = parts
                vocab_dict[index] = word  # Store mapping
    return vocab_dict

# Function to decode category and question only
def decode_questions(encoded_file_path, vocab_dict1, vocab_dict2):
    decoded_questions = []
    
    try:
        with gzip.open(encoded_file_path, 'rt', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split("\t")
                
                if len(parts) >= 2:  # Ensure the format is correct
                    question_category = parts[0]  # The first part is the category
                    encoded_question = parts[1]  # The second part is the encoded question

                    # Decode the question using both vocab dictionaries
                    decoded_question = " ".join([vocab_dict1.get(token, vocab_dict2.get(token, "[UNKNOWN]")) for token in encoded_question.split()])

                    # Store the decoded category and question
                    decoded_questions.append({
                        'category': question_category,
                        'question': decoded_question
                    })
    except Exception as e:
        print(f"Error decoding file {encoded_file_path}: {e}")
    
    return decoded_questions

# File paths
vocab_file1_path = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\vocabulary"
vocab_file2_path = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\vocabulary.txt"
train_file_path = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.train.encoded.gz"
test1_file_path = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.test.encoded.gz"
test2_file_path = r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1000.pool.solr.test.encoded.gz"

# Load both vocabulary files
vocab_dict1 = load_vocabulary(vocab_file1_path)
vocab_dict2 = load_vocabulary(vocab_file2_path)

# Decode only questions
train_questions = decode_questions(train_file_path, vocab_dict1, vocab_dict2)
test1_questions = decode_questions(test1_file_path, vocab_dict1, vocab_dict2)
test2_questions = decode_questions(test2_file_path, vocab_dict1, vocab_dict2)

# Combine all decoded questions
all_questions_decoded = train_questions + test1_questions + test2_questions

# Save decoded data as a JSON file
output_file_path = r"D:\NLPInsuranceProject\decoded_questions.json"
with open(output_file_path, 'w', encoding='utf-8') as f_out:
    json.dump(all_questions_decoded, f_out, indent=4, ensure_ascii=False)

print(f"✅ Decoding complete! Results saved to {output_file_path}")

# Display a few decoded examples
print(json.dumps(all_questions_decoded[:3], indent=4, ensure_ascii=False))


✅ Decoding complete! Results saved to D:\NLPInsuranceProject\decoded_questions.json
[
    {
        "category": "disability-insurance",
        "question": "Is Disability Insurance Required By Law?"
    },
    {
        "category": "life-insurance",
        "question": "Can Creditors Take Life Insurance After Death?"
    },
    {
        "category": "renters-insurance",
        "question": "Does Travelers Insurance Have Renters Insurance?"
    }
]


In [2]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


# HE WALA GHE

In [1]:
import sqlite3
import speech_recognition as sr
import pyttsx3
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown

# ✅ Set Up OpenRouter API
API_KEY = "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" #"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"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
    default_headers={ 
        "Authorization": f"Bearer {API_KEY}",  
        "X-Title": "Insurance Chatbot"
    }
)

In [4]:
import json
from sentence_transformers import InputExample

# ✅ Load the JSON File
with open("decoded_questions.json", "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# ✅ Extract question pairs (create synthetic paraphrases for now)
examples = []
questions = [entry["question"] for entry in questions_data]

# We'll use simple random pairs (not ideal, but okay for initial fine-tune)
for i in range(len(questions) - 1):
    examples.append(InputExample(texts=[questions[i], questions[i + 1]]))

print(f"Total training pairs: {len(examples)}")


Total training pairs: 16888


In [5]:
import json
from sentence_transformers import InputExample
from collections import defaultdict
import random

# ✅ Load decoded questions
with open("decoded_questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Group questions by category
category_to_questions = defaultdict(list)
for item in data:
    category = item["category"]
    question = item["question"]
    category_to_questions[category].append(question)

# ✅ Create pairs of similar questions from the same category
examples = []
for category, questions in category_to_questions.items():
    if len(questions) < 2:
        continue
    random.shuffle(questions)
    for i in range(0, len(questions) - 1, 2):
        q1, q2 = questions[i], questions[i + 1]
        examples.append(InputExample(texts=[q1, q2]))

print(f"✅ Prepared {len(examples)} training examples")


✅ Prepared 8441 training examples


In [6]:
sentences1 = [
    "What is term insurance?",
    "How can I cancel my health insurance?",
    "Does motor insurance cover theft?"
]

sentences2 = [
    "Explain term insurance coverage.",
    "What’s the process for cancelling a policy?",
    "Is vehicle theft included in insurance?"
]

scores = [0.95, 0.8, 0.9]  # Based on your judgment or annotations


In [8]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator


# ✅ Load base model
model = SentenceTransformer("all-MiniLM-L6-v2")
#model = model.to('cuda')

# ✅ Prepare DataLoader
train_dataloader = DataLoader(examples, shuffle=True, batch_size=64)

# ✅ Choose Loss
train_loss = losses.MultipleNegativesRankingLoss(model)

evaluator = EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    evaluator=evaluator,
    evaluation_steps=500,
    warmup_steps=100,
    show_progress_bar=True
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine
132,No log,No log,0.975155,1.000000
264,No log,No log,0.972147,1.000000
396,No log,No log,0.971722,1.000000
500,2.305000,No log,0.969061,1.000000
528,2.305000,No log,0.971763,1.000000
660,2.305000,No log,0.970769,1.000000


In [9]:
model_save_path = "custom_insurance_encoder"
model.save(model_save_path)
print(f"✅ Model saved at: {model_save_path}")


✅ Model saved at: custom_insurance_encoder


In [10]:
sentences = [
    "What does liability insurance cover?",
    "Tell me about car insurance coverage.",
    "When does a policy lapse?"
]

embeddings = model.encode(sentences, convert_to_tensor=True)

# Check cosine similarity (optional)
from sentence_transformers.util import cos_sim
print("Similarity between 1st and 2nd:", cos_sim(embeddings[0], embeddings[1]))


Similarity between 1st and 2nd: tensor([[0.8101]])


# HE WALA GHE

In [2]:
from IPython.display import display, Markdown


# ✅ Load Fine-Tuned Sentence Transformer
model = SentenceTransformer(r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\custom_insurance_encoder")

# ✅ Load Insurance Questions
import json
with open(r"D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\decoded_questions.json", "r", encoding="utf-8") as file:
    insurance_data = json.load(file)

questions = [entry["question"] for entry in insurance_data]
question_embeddings = model.encode(questions)

# ✅ Set Up SQLite for Memory
conn = sqlite3.connect("chatbot_memory.db", check_same_thread=False)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS chatbot_memory (query TEXT, response TEXT)")
conn.commit()

# ✅ Function to Search Memory
def search_memory(user_query):
    cursor.execute("SELECT response FROM chatbot_memory WHERE query=?", (user_query,))
    result = cursor.fetchone()
    return result[0] if result else None

# ✅ Function to Store in Memory
def store_memory(user_query, response):
    cursor.execute("INSERT INTO chatbot_memory (query, response) VALUES (?, ?)", (user_query, response))
    conn.commit()

# ✅ Function for Semantic Search
def find_relevant_questions(user_query):
    query_embedding = model.encode([user_query])
    similarities = np.dot(query_embedding, question_embeddings.T)[0]
    top_indices = np.argsort(similarities)[-3:][::-1]
    relevant_questions = [questions[i] for i in top_indices]
    return relevant_questions

# ✅ Function to Get Insurance Answer
def get_insurance_response(user_query):
    # Step 1: Check Memory
    memory_response = search_memory(user_query)
    if memory_response:
        return f"[From Memory] {memory_response}"

    # Step 2: Find Relevant Questions
    relevant_questions = find_relevant_questions(user_query)

    # Step 3: Call OpenRouter (DeepSeek R1)
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free", #"deepseek/deepseek-v3-base:free",  #
            messages=[
                {"role": "system", "content": "You are an insurance chatbot based from India who follows rules and regulations related to insurance for India. Answer insurance-related questions only. If non-insurance related questions asked please politely deny saying i am not made for this domain"},
                {"role": "user", "content": f"User Query: {user_query}\nRelevant Questions: {relevant_questions}"}
            ],
            temperature=0.3,
        )
        chatbot_response = response.choices[0].message.content
        store_memory(user_query, chatbot_response)  # Save response in memory
        return chatbot_response
    except Exception as e:
        return f"Error: {str(e)}"

# ✅ Voice Input & Output Functions
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return "Sorry, I couldn't understand."
    except sr.RequestError:
        return "Speech recognition service is unavailable."

def speak_response(response):
    engine = pyttsx3.init()
    engine.say(response)
    
    try:
        engine.runAndWait()  # Normal execution
    except RuntimeError:
        engine.endLoop()  # Stop the current event loop
        engine.runAndWait() 

# ✅ Main Chat Loop (Conversation Mode)
def start_chatbot():
    print("💬 Insurance Chatbot Started! Type 'voice' for voice input. Say 'thank you', 'ok', or 'exit' to stop.")
    
    while True:
        user_input = input("👤 You: ").strip().lower()
        print("User question: " ,user_input)

        # Handle voice input
        if user_input == "voice":
            user_input = recognize_speech()
            print(f"👤 You (via voice): {user_input}")

        # Check for exit words
        if user_input in ["thank you", "thanks", "ok", "exit", "goodbye", "bye"]:
            print("🤖 Bot: You're welcome! Have a great day! 😊")
            speak_response("You're welcome! Have a great day!")
            break

        # Get response
        bot_response = get_insurance_response(user_input)
    
        display(Markdown(f"🤖 Bot: {bot_response}"))
        speak_response(bot_response)

        # Ask if further help is needed
        follow_up = input("🤖 Bot: Do you need help with anything else? (yes/no) ").strip().lower()
        if follow_up in ["no", "thank you", "thanks", "ok", "exit", "goodbye", "bye"]:
            print("🤖 Bot: Have a great day! 😊")
            speak_response("Have a great day!")
            break

# ✅ Run the Chatbot
if __name__ == "__main__":
    start_chatbot()


💬 Insurance Chatbot Started! Type 'voice' for voice input. Say 'thank you', 'ok', or 'exit' to stop.
User question:  what are the insurance companies present in india


🤖 Bot: In India, the insurance sector is regulated by the **Insurance Regulatory and Development Authority of India (IRDAI)**. The industry includes **public-sector (government-owned)** and **private-sector companies**, categorized into **life insurance**, **general (non-life) insurance**, **health insurance**, and **reinsurance**. Here’s a categorized list of key players:

### **Public Sector Insurers**
1. **Life Insurance**  
   - Life Insurance Corporation of India (LIC)  

2. **General Insurance**  
   - New India Assurance  
   - United India Insurance  
   - National Insurance Company  
   - Oriental Insurance Company  

3. **Reinsurance**  
   - General Insurance Corporation of India (GIC Re)  

---

### **Private Sector Insurers**
1. **Life Insurance**  
   - HDFC Life  
   - SBI Life  
   - ICICI Prudential Life  
   - Max Life  
   - Bajaj Allianz Life  
   - Kotak Mahindra Life  
   - Aditya Birla Sun Life  

2. **General Insurance**  
   - ICICI Lombard  
   - Bajaj Allianz General  
   - Tata AIG  
   - Reliance General  
   - Cholamandalam MS  
   - Future Generali  

3. **Standalone Health Insurance**  
   - Star Health and Allied Insurance  
   - Niva Bupa Health Insurance  
   - Care Health Insurance  
   - ManipalCigna  

4. **Specialized Insurers**  
   - Agriculture Insurance Company (crop insurance)  
   - Export Credit Guarantee Corporation (ECGC)  

---

### **Note**  
- This is a **non-exhaustive list**. For a complete list of IRDAI-registered insurers, visit [IRDAI's official website](https://www.irdai.gov.in).  
- Always verify a company’s license and product suitability before purchasing a policy.  

Let me know if you need details about specific insurers! 😊

🤖 Bot: Have a great day! 😊


## Eval metrics

In [3]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
import pandas as pd


In [5]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

def get_insurance_response(user_query):
    # Step 1: Check Memory
    memory_response = search_memory(user_query)
    if memory_response:
        return f"[From Memory] {memory_response}"

    # Step 2: Find Relevant Questions
    relevant_questions = find_relevant_questions(user_query)

    # Step 3: Call OpenRouter (DeepSeek R1)
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free", #"deepseek/deepseek-v3-base:free",  #
            messages=[
                {"role": "system", "content": "You are an insurance chatbot based from India who follows rules and regulations related to insurance for India. Answer insurance-related questions only. If non-insurance related questions asked please politely deny saying i am not made for this domain"},
                {"role": "user", "content": f"User Query: {user_query}\nRelevant Questions: {relevant_questions}"}
            ],
            temperature=0.3,
        )
        chatbot_response = response.choices[0].message.content
        store_memory(user_query, chatbot_response)  # Save response in memory
        return chatbot_response
    except Exception as e:
        return f"Error: {str(e)}"

# 👉 Your LLM-based evaluator (replace with working agent call)
def evaluate_response_with_agent(user_query, predicted_response):
    try:
        eval_prompt = (
            f"You are an expert insurance evaluator in the Indian domain. "
            f"Evaluate the chatbot's response to the user query based on 3 factors:\n"
            f"1. Helpfulness: Does it answer the query?\n"
            f"2. Clarity: Is it easy to understand?\n"
            f"3. Accuracy: Is the information correct as per Indian insurance rules and practices?\n\n"
            f"Query: {user_query}\n"
            f"Response: {predicted_response}\n\n"
            f"Give a rating from 1 (Very Poor) to 5 (Excellent). Only return the rating as a number."
        )

        eval_response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {"role": "system", "content": "You are a strict evaluation agent scoring chatbot responses from 1 to 5 only."},
                {"role": "user", "content": eval_prompt}
            ],
            temperature=0.1
        )
        score = eval_response.choices[0].message.content.strip()
        return score
    except Exception as e:
        return f"Error: {str(e)}"

# ✅ 1. Define Test Queries
test_queries = [
    "How do I file a car insurance claim online?",
    "What documents are needed for a health insurance claim?",
    "How can I pay my life insurance premium via UPI?",
    "How do I add a family member to my health insurance policy?",
    "What riders can I add to my term insurance plan?",
    "How do I update my nominee details?",
    "Can you explain the waiting period in health insurance in simple terms?",
    "Is life insurance mandatory in India?",
    "What are the tax benefits of health insurance?",
    "Can I transfer my bike insurance to a new owner?"
]

# ✅ 2. Define Reference Answers (You already have these)
reference_answers = [ 
    "To file a car insurance claim online in India, follow these general steps: Visit your insurer’s official website or mobile app. Log in using your policy details or registered mobile number. Go to the ‘Claims’ section and select ‘Motor/Car Insurance Claim’. Fill out the claim form by providing details like: Date and time of the incident, Location of the accident, Vehicle number and policy number, Description of the damage or loss. Upload required documents, which may include: Copy of your insurance policy, FIR (if applicable), Photos of the vehicle damage, Driving license and RC book. Submit the claim. You’ll receive a claim reference number for tracking. Wait for inspection. The insurer may assign a surveyor to inspect the vehicle. Claim approval and settlement. Once verified, you’ll be notified about the approval and the repair process or reimbursement. Note: Always check with your specific insurer as processes may vary slightly.",
    
    "To file a health insurance claim in India, you typically need the following documents: Duly filled and signed claim form (cashless or reimbursement, as applicable); Health insurance policy copy or e-card; Photo ID proof (Aadhaar, PAN, Voter ID, etc.); Doctor's prescription and consultation papers; Hospital bills and receipts (original copies); Discharge summary or hospital discharge certificate; Diagnostic test reports (blood test, X-ray, MRI, etc.); Pharmacy bills with prescriptions; FIR or Medico-Legal Certificate (MLC) - in case of an accident or medico-legal case; Cancelled cheque or bank account details for reimbursement. Note: Requirements may vary slightly between insurers. Always check with your insurance provider.",

    "To pay your life insurance premium via UPI in India, follow these steps: 1. Open your UPI app (Google Pay, PhonePe, Paytm, BHIM, etc.) 2. Select Insurance or Pay Bills section (varies by app) 3. Choose your insurance provider from the list 4. Enter your policy number and verify details 5. Enter the premium amount 6. Select your UPI payment method (linked bank account) 7. Authenticate with your UPI PIN 8. Save the payment confirmation for your records. Note: Some insurers allow direct payment through their app/website via UPI. Always verify the payment recipient name matches your insurer. Payment may take 24-48 hours to reflect in your policy.",

    "You can add a family member to your health insurance policy by: 1. Logging into your insurer's portal/app or calling customer care 2. Requesting a family addition endorsement 3. Submitting these documents: Policyholder's ID proof, Family member's ID (Aadhaar/PAN), Relationship proof (birth/marriage certificate), Recent photo of the member 4. Paying any extra premium required. Typically takes 3-7 working days to process. 30-day waiting period usually applies for new additions.",

    "Common riders you can add to your term insurance plan in India: Accidental Death Benefit Rider; Critical Illness Rider; Permanent Disability Rider; Waiver of Premium Rider; Income Benefit Rider. Note: Rider availability varies by insurer. Additional premium applies for each rider. Check your policy terms for exact coverage details.",

    "To update nominee details: 1. Visit your insurer's website/app or branch office 2. Fill the Nomination Change form 3. Submit: Policy document, New nominee's ID proof, Your ID proof, Cancelled cheque (if changing bank details) 4. Get acknowledgement receipt. Changes take 3-5 working days. You can update nominees multiple times during policy term.",

    "Health insurance waiting period means: Initial Waiting Period: 30-90 days (no claims allowed); Pre-existing Disease: 1-4 years (for conditions before buying policy); Specific Treatments: 1-2 years (like hernia or cataract surgery). Example: If hospitalized within 15 days of purchase, claim is usually denied unless due to accident.",

    "No, life insurance is not mandatory in India by law. However, some employers make it mandatory for employees. Home loan providers may require term insurance. It's advisable if you have dependents. Only motor insurance is legally mandatory under the Motor Vehicles Act.",

    "Health insurance tax benefits under Section 80D: ₹25,000 deduction for self/family (₹50,000 for senior citizens); Additional ₹25,000 for parents (₹50,000 if seniors); ₹5,000 for preventive health check-ups (included in above). Max deduction: ₹1,00,000 for family + senior parents.",

    "Yes, you can transfer bike insurance to new owner. 1. Inform insurer within 14 days of sale. 2. Submit RC transfer copy, New owner's details, NOC from seller. 3. Pay applicable charges. New owner gets remaining policy. Comprehensive policies are transferable; third-party may not be mid-term."
]

# ✅ 3. Initialize Evaluators
smoothie = SmoothingFunction().method4
rouge = Rouge()

results = []

# ✅ 4. Evaluation Loop
for idx, query in enumerate(test_queries):
    predicted = get_insurance_response(query)
    reference = reference_answers[idx]

    bleu_score = sentence_bleu([reference.split()], predicted.split(), smoothing_function=smoothie) if reference else None
    rouge_scores = rouge.get_scores(predicted, reference)[0] if reference else {}
    agent_score = evaluate_response_with_agent(query, predicted)

    results.append({
        "Query": query,
        "Predicted Answer": predicted,
        "Reference Answer": reference,
        "BLEU Score": round(bleu_score, 4) if bleu_score is not None else None,
        "ROUGE-L Precision": round(rouge_scores.get("rouge-l", {}).get("p", 0), 4),
        "ROUGE-L Recall": round(rouge_scores.get("rouge-l", {}).get("r", 0), 4),
        "ROUGE-L F1": round(rouge_scores.get("rouge-l", {}).get("f", 0), 4),
        "LLM Agent Score": agent_score
    })

# ✅ 5. Save Results to CSV
df = pd.DataFrame(results)
df.to_csv("chatbot_evaluation_results.csv", index=False)
print("✅ Evaluation complete. Results saved to chatbot_evaluation_results.csv.")


✅ Evaluation complete. Results saved to chatbot_evaluation_results.csv.


In [6]:
df

,Query,Predicted Answer,Reference Answer,BLEU Score,ROUGE-L Precision,ROUGE-L Recall,ROUGE-L F1,LLM Agent Score
0,How do I file a car insurance claim online?,"To file a car insurance claim online in India,...","To file a car insurance claim online in India,...",0.0998,0.2690,0.3679,0.3108,4
1,What documents are needed for a health insuran...,**Documents Required for a Health Insurance Cl...,"To file a health insurance claim in India, you...",0.0717,0.2308,0.3667,0.2833,5
2,How can I pay my life insurance premium via UPI?,To pay your life insurance premium via UPI in ...,To pay your life insurance premium via UPI in ...,0.0942,0.2953,0.5301,0.3793,5
3,How do I add a family member to my health insu...,To add a family member to your health insuranc...,You can add a family member to your health ins...,0.0464,0.1548,0.3810,0.2202,4
4,What riders can I add to my term insurance plan?,"In India, you can enhance your term insurance ...",Common riders you can add to your term insuran...,0.0310,0.1982,0.5238,0.2876,5
5,How do I update my nominee details?,To update your nominee details on an insurance...,To update nominee details: 1. Visit your insur...,0.0067,0.1135,0.3200,0.1675,5
6,Can you explain the waiting period in health i...,**Answer:** \nThe **waiting period** in healt...,Health insurance waiting period means: Initial...,0.0059,0.0730,0.2222,0.1099,5
7,Is life insurance mandatory in India?,"**Answer:** \nNo, life insurance is **not man...","No, life insurance is not mandatory in India b...",0.0386,0.1856,0.4865,0.2687,5
8,What are the tax benefits of health insurance?,"Under Section 80D of the Income Tax Act, healt...",Health insurance tax benefits under Section 80...,0.0060,0.0800,0.4000,0.1333,5
9,Can I transfer my bike insurance to a new owner?,"In India, **you cannot directly transfer your ...","Yes, you can transfer bike insurance to new ow...",0.0084,0.1522,0.4884,0.2320,5
